## Imports

In [73]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [74]:
df = pd.read_csv('redfin_2023-05-12-09-18-46.csv')

## Data Clean

In [75]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [76]:
df = df.dropna(subset=['SOLD DATE'])

In [77]:
# Desired month
desired_month = 'April'

# Define list of desired months (excluding current month)
desired_months = [desired_month]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [78]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    1739
Name: PRICE, dtype: int64
-------
False    1267
True      472
Name: $/SQUARE FEET, dtype: int64
-------
False    1593
True      146
Name: YEAR BUILT, dtype: int64
-------


In [79]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [80]:
# df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
# df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
# df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
# df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
# df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [81]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1695,PAST SALE,April-28-2023,Condo/Co-op,4300 W Ford City Dr Unit A303,Chicago,IL,60652.0,19000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/4300-W-Ford-...,MRED,11395026.0,N,Y,41.753941,-87.730729
1319,PAST SALE,April-20-2023,Condo/Co-op,8018 S Hoyne Ave #4,Chicago,IL,60620.0,40000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/8018-S-Hoyne...,MRED,11717344.0,N,Y,41.747673,-87.675768
1686,PAST SALE,April-13-2023,Condo/Co-op,15319 Chicago Rd #8,Dolton,IL,60419.0,43000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Dolton/15319-Doctor-...,MRED,11622725.0,N,Y,41.616138,-87.606710
183,PAST SALE,April-3-2023,Condo/Co-op,1249 Cunningham Dr Unit 1W,Calumet City,IL,60409.0,43500.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/IL/Calumet-City/1249-Cu...,MRED,11723020.0,N,Y,41.599691,-87.551271
835,PAST SALE,April-7-2023,Condo/Co-op,10535 S Hale Ave Unit 3E,Chicago,IL,60643.0,48500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/10535-S-Hale...,MRED,11699921.0,N,Y,41.701891,-87.668705
1310,PAST SALE,April-4-2023,Condo/Co-op,800 E 191st Pl #402,Glenwood,IL,60425.0,50000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Glenwood/800-E-191st...,MRED,11708684.0,N,Y,41.542097,-87.592521
201,PAST SALE,April-24-2023,Condo/Co-op,12558 S Fairview Ave Unit 3-A,Blue Island,IL,60406.0,55000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Blue-Island/12558-Fa...,MRED,11715002.0,N,Y,41.664808,-87.689262
272,PAST SALE,April-12-2023,Condo/Co-op,7337 S South Shore Dr #708,Chicago,IL,60649.0,57000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/7337-S-South...,MRED,11726595.0,N,Y,41.763318,-87.558357
301,PAST SALE,April-10-2023,Condo/Co-op,100 Park Ave #307,Calumet City,IL,60409.0,59000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Calumet-City/100-Par...,MRED,11743143.0,N,Y,41.598254,-87.572815
1733,PAST SALE,April-19-2023,Condo/Co-op,18550 Torrence Ave #4,Lansing,IL,60438.0,60000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Lansing/18550-Torren...,MRED,11388170.0,N,Y,41.558166,-87.559091


In [82]:
print(df_filtered['URL'].iloc[416])

https://www.redfin.com/IL/Chicago/1350-W-Grand-Ave-60642/unit-2C/home/12813474


In [83]:
# # Correct the prices, if needed
# df_filtered.at[618,'PRICE']=(295000)
# df_filtered.at[832,'PRICE']=(355000)

In [84]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [85]:
# # Corrections, if needed
# df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
# df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [86]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
1695,4300 W Ford City Dr Unit A303,Chicago,22.0
1194,100 Park Ave #700,Calumet City,59.0
1483,4300 W Ford City Dr Unit A905,Chicago,59.0
1310,800 E 191st Pl #402,Glenwood,63.0
1733,18550 Torrence Ave #4,Lansing,63.0
1164,400 Park Ave #709,Calumet City,65.0
186,8136 S Calumet Ave Unit 2S,Chicago,67.0
1686,15319 Chicago Rd #8,Dolton,69.0
639,18224 Wentworth Ave #5,Lansing,71.0
1319,8018 S Hoyne Ave #4,Chicago,73.0


In [87]:
print(df_filtered.URL.iloc[217])

https://www.redfin.com/IL/Barrington/422-W-Russell-St-60010/unit-1/home/18933991


In [88]:
# Corrections, if needed
#
#
#

In [89]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [90]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [91]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [92]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [93]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [94]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [95]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{desired_month} 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [96]:
m.save('index.html')

## Summary Info

In [97]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [98]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [99]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [100]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
CHI - Near North Side, 159 E Walton Pl Unit 32A Chicago | Price $5,200,000 | $965 psf | Year built: 1928
Least Expensive
CHI - West Lawn, 4300 W Ford City Dr Unit A303 Chicago | Price $19,000 | $22 psf | Year built: 1970
Highest Price Per Square Foot
CHI - South Shore, 6730 S South Shore Dr #505 Chicago | Price $825,000 | $1,122 psf | Year built: 1964
Lowest Price Per Square Foot
CHI - West Lawn, 4300 W Ford City Dr Unit A303 Chicago | Price $19,000 | $22 psf | Year built: 1970
Newest
CHI - Lincoln Park, 1719 N Burling St Unit 4N Chicago | Price $1,850,000 | $nan psf | Year built: 2023
Oldest
CHI - West Town, 1647 W North Ave #1 Chicago | Price $372,000 | $345 psf | Year built: 1876


## Time on Market Calculator

In [101]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [102]:
# from datetime import datetime, timedelta

# date1 = datetime(2022, 12, 15) ## List (Earlier) date
# date2 = datetime(2023, 3, 30) ## Close (Later) date

# delta = date2 - date1
# num_days = delta.days

# print(num_days)

## Map URL Snagger

In [103]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [104]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCounty_condo_sales_month_ending_april_2023


## Get Summary Data

In [105]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 1739
--------
Total sale price: $660,324,466
--------
Median sale price: $285,500
--------
Max sale price: $5,200,000
--------
Min sale price: $19,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,122
--------
Min price per square foot: $22
--------
Median price per square foot: $237
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1876.0
----------
Average building age: 1977.668549905838


## To CSV

In [106]:
df_filtered.to_csv(f'{desired_month}_condo.csv')